In [ ]:
%matplotlib inline

In [2]:
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
from matplotlib import rcParams

import numpy as np
import pandas as pd


import shapely.geometry as sgeom

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

import dask.dataframe as dd
import dask.bag as db
import dask.diagnostics as dg

import geopandas as gpd


paper_path = Path('../../paper/figures/')

slide_path = Path('../../slides/figures/intro/')
proposal_path = Path('../../draft/figures/intro/')
#rcParams['font.family'] = 'Segoe Print'

C:\Users\story\miniconda3\envs\paper\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [3]:
YEAR = 2021
names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
ds = dd.read_csv(f's3://noaa-ghcn-pds/csv/{YEAR}.csv', storage_options={'anon':True},  names=names, memory_map=False, 
                  dtype={'DATA_VALUE':'object'}, parse_dates=['DATE', 'OBS-TIME'])

ImportError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: Install s3fs to access S3

In [ ]:
# {column name:extents of the fixed-width fields}
columns = {"ID": (0,11), "LATITUDE": (12, 20), "LONGITUDE": (21, 30), "ELEVATION": (31, 37),"STATE": (38, 40),
           "NAME": (41, 71), "GSN FLAG": (72, 75), "HCN/CRN FLAG": (76, 79),"WMO ID": (80, 85)}

In [ ]:
df = pd.read_fwf("http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt", 
                    colspecs=list(columns.values()), names=list(columns.keys())).dropna(subset=['STATE'])

In [ ]:
df['STATE'].unique()

In [ ]:
nydf = df[df['STATE'].str.match("")]

In [ ]:
['']

In [ ]:
ds['ELEMENT'].unique().compute()

In [ ]:
nyds = ds[ds['ID'].isin(nydf['ID'].unique()) & ds['ELEMENT'].isin(['TAVG', 'PRCP' ])].compute()

In [ ]:
ny = nyds.merge(nydf, on='ID')

In [ ]:
ny.columns

In [ ]:
ds['ELEMENT'].unique().compute()

In [ ]:
nyd = ny.pivot(index=['NAME','LATITUDE', 'LONGITUDE', 'DATE'], columns=['ELEMENT'], values=['DATA_VALUE']).reset_index()

In [ ]:
nyd

In [ ]:
nyd.columns = [c[0] if i <4 else c[1] for i, c in enumerate(nyd.columns.to_flat_index())]

In [ ]:
nyd.head()

In [ ]:
nyd['TAVGF'] =  (nyd['TAVG'].astype(float)/10) *(9/5) + 32

In [ ]:
nyd.head()

In [ ]:
nyd['PRCP'].astype(float).describe()

In [ ]:
nyd['PRCPI'] = nyd['PRCP'].astype(float)/10 * 0.039370 

In [ ]:
nyd['PRCPI'].describe()

In [ ]:
nyd.to_csv("nydata", index=False)

In [2]:
%conda install dask

error: incomplete escape \U at position 28